In [ ]:
# ------------------------------------------------------------------------------
# This is auto generated  
# To run this code, please install SDK by this command:
# !pip install "azure-ml-component[notebooks]" --extra-index-url https://azuremlsdktestpypi.azureedge.net/modulesdkpreview --upgrade
# More detailed guide to set up your environment: https://github.com/Azure/DesignerPrivatePreviewFeatures/blob/master/azure-ml-components/samples/setup-environment.ipynb
# ------------------------------------------------------------------------------

In [ ]:
from azureml.core import Workspace
from azure.ml.component import Pipeline, Component, dsl

In [ ]:
# configure aml workspace
ws = Workspace.from_config()

In [ ]:
# get components
azureml_score_model_func, azureml_evaluate_model_func, azureml_normalize_data_func, azureml_split_data_func, azureml_clean_missing_data_func, azureml_decision_forest_regression_func, azureml_tune_model_hyperparameters_func = Component.batch_load(ws, selectors=['azureml://Score Model', 'azureml://Evaluate Model', 'azureml://Normalize Data', 'azureml://Split Data', 'azureml://Clean Missing Data', 'azureml://Decision Forest Regression', 'azureml://Tune Model Hyperparameters'])

In [ ]:
# get dataset
from azureml.core import Dataset
training = Dataset.get_by_name(ws, name='training', version=1)

In [ ]:
# define pipeline
@dsl.pipeline(name='pipeline hyperparametres', description='Pipeline-Created-on-07-30-2021', default_compute_target='jcharley1', default_datastore='workspaceblobstore')
def generated_pipeline():
    azureml_clean_missing_data_0 = azureml_clean_missing_data_func(
        dataset=training,
        columns_to_be_cleaned='[{"KeepInputDataOrder":true,"ColumnNames":["Access_Level"]}]',
        minimum_missing_value_ratio=0.0,
        maximum_missing_value_ratio=1.0,
        cleaning_mode='Replace with mean',
        generate_missing_value_indicator_column=False,
        cols_with_all_missing_values='Remove',
        replacement_value='0')
    azureml_clean_missing_data_0.runsettings.resource_layout.configure(node_count=1)
    azureml_clean_missing_data_0.runsettings.docker_configuration.configure(use_docker=True, shared_volumes=True, shm_size='2g', arguments='[]')
    
    azureml_normalize_data_0 = azureml_normalize_data_func(
        dataset=azureml_clean_missing_data_0.outputs.cleaned_dataset,
        transformation_method='ZScore',
        use_0_for_constant_columns_when_checked=True,
        columns_to_transform='[{"KeepInputDataOrder":true,"ColumnNames":["peerUsageMetric6"]}]')
    azureml_normalize_data_0.runsettings.resource_layout.configure(node_count=1)
    azureml_normalize_data_0.runsettings.docker_configuration.configure(use_docker=True, shared_volumes=True, shm_size='2g', arguments='[]')
    
    azureml_split_data_0 = azureml_split_data_func(
        dataset=azureml_normalize_data_0.outputs.transformed_dataset,
        splitting_mode='Split Rows',
        fraction_of_rows_in_the_first_output_dataset=0.7,
        randomized_split=True,
        random_seed=0,
        stratified_split='True',
        stratification_key_column='[{"KeepInputDataOrder":true,"ColumnNames":["EmployeeTargeted"]}]',
        regular_expression='\"column name" ^start',
        relational_expression='\"column name" > 3')
    azureml_split_data_0.runsettings.resource_layout.configure(node_count=1)
    azureml_split_data_0.runsettings.docker_configuration.configure(use_docker=True, shared_volumes=True, shm_size='2g', arguments='[]')
    
    azureml_decision_forest_regression_0 = azureml_decision_forest_regression_func(
        create_trainer_mode='SingleParameter',
        number_of_decision_trees=8,
        maximum_depth_of_the_decision_trees=32,
        minimum_number_of_samples_per_leaf_node=1,
        resampling_method='Bagging Resampling',
        range_for_number_of_decision_trees='1; 8; 32',
        range_for_the_maximum_depth_of_the_decision_trees='1; 16; 64',
        range_for_the_minimum_number_of_samples_per_leaf_node='1; 4; 16')
    azureml_decision_forest_regression_0.runsettings.resource_layout.configure(node_count=1)
    azureml_decision_forest_regression_0.runsettings.docker_configuration.configure(use_docker=True, shared_volumes=True, shm_size='2g', arguments='[]')
    
    azureml_tune_model_hyperparameters_0 = azureml_tune_model_hyperparameters_func(
        untrained_model=azureml_decision_forest_regression_0.outputs.untrained_model,
        training_dataset=azureml_split_data_0.outputs.results_dataset1,
        specify_parameter_sweeping_mode='Entire grid',
        name_or_numerical_index_of_the_label_column='[{"KeepInputDataOrder":true,"ColumnNames":["EmployeeTargeted"]}]',
        metric_for_measuring_performance_for_classification='Accuracy',
        metric_for_measuring_performance_for_regression='Mean absolute error',
        maximum_number_of_runs_on_random_sweep=5,
        random_seed=0)
    azureml_tune_model_hyperparameters_0.runsettings.resource_layout.configure(node_count=1)
    azureml_tune_model_hyperparameters_0.runsettings.docker_configuration.configure(use_docker=True, shared_volumes=True, shm_size='2g', arguments='[]')
    
    azureml_score_model_0 = azureml_score_model_func(
        trained_model=azureml_tune_model_hyperparameters_0.outputs.trained_best_model,
        dataset=azureml_split_data_0.outputs.results_dataset2,
        append_score_columns_to_output=True)
    azureml_score_model_0.runsettings.resource_layout.configure(node_count=1)
    azureml_score_model_0.runsettings.docker_configuration.configure(use_docker=True, shared_volumes=True, shm_size='2g', arguments='[]')
    
    azureml_evaluate_model_0 = azureml_evaluate_model_func(
        scored_dataset=azureml_score_model_0.outputs.scored_dataset)
    azureml_evaluate_model_0.runsettings.resource_layout.configure(node_count=1)
    azureml_evaluate_model_0.runsettings.docker_configuration.configure(use_docker=True, shared_volumes=True, shm_size='2g', arguments='[]')

In [ ]:
# create a pipeline
pipeline = generated_pipeline()

In [ ]:
# validate pipeline and visualize the graph
pipeline.validate()

In [ ]:
# submit a pipeline run
# pipeline.submit(experiment_name='sample-experiment-name').wait_for_completion()